# Лабораторна робота 2 з "Асиметричних криптосистем та протоколів"
## Тема: Вивчення криптосистеми RSA та алгоритму електронного підпису; ознайомлення з методами генерації параметрів для асиметричних криптосистем

**Виконали**\
Дигас Богдан, ФІ-03\
Починок Юрій, ФІ-03

In [229]:
import random
import math
import numpy as np
rand = random.SystemRandom()

## Miller-Rabin test

In [230]:

def decomposing_number(n, a):
    exp = n - 1
    while not exp & 1:  # while exp is even
        exp >>= 1  # divide by 2
    if pow(a, exp, n) == 1:
        return True  # number is composite
    while exp < n - 1:
        if pow(a, exp, n) == n - 1:
            return True  # number is composite
        exp <<= 1  # multiply by 2
    return False  # number is probably prime


def miller_rabbin_test(n, k=20):
    for i in range(k):
        a = rand.randrange(1, n - 1)
        if not decomposing_number(n, a):
            return False  # number is composite
    return True  # number is probably prime

def bin_to_dec(bin_n):
    dec_n = 0
    res = 0
    for i in range(len(bin_n)):
        res = bin_n[len(bin_n) - i - 1] * 2 ** i
        dec_n += res
    return dec_n


## Prime number generation

In [231]:
def generate_bit_seq(n):
    seq = [0]*n
    for i in range(n):
        seq[i] = rand.randint(0, 1)
    return seq


def L20(n):
    seq = generate_bit_seq(20)
    result = [0]*n
    for i in range(20):
        result[i] = seq[i]
    for i in range(20,n):
        result[i] = result[i-3]^result[i-5]^result[i-9]^result[i-20]
    return result


In [232]:
def generate_prime_number(x):
    res = [1,0,0]
    while(miller_rabbin_test(bin_to_dec(res)) == False):
        res = L20(x)
    return res

## Better prime number generation 2*k*p+1

In [234]:
def generate_better_prime_numbers(n):
    p = bin_to_dec(generate_prime_number(n))
    q = bin_to_dec(generate_prime_number(n))
    
    i = 1
    while(miller_rabbin_test(2*p*i + 1) == False):
        i = i + 1   

    better_p = 2*p*i + 1
    
    j = 1
    while(miller_rabbin_test(2*q*j + 1) == False):
        j = j + 1

    better_q = 2*q*j + 1
    
    return better_p, better_q

## Initialization 

In [235]:
def RSA_gen_constants():
    p,q = generate_better_prime_numbers(256)
    e = 2**16 + 1
    n = p*q
    phi_n = (p-1)*(q-1)
    d = pow(e,-1,phi_n)
    
    return p,q,n,e,d

## User and his parameters

In [236]:
class User:
    
    __private_key = None
    __key_pair = None
    
    __k = None

    def __init__(self):
        result = RSA_gen_constants()
        self.__key_pair = result[0], result[1]
        self.public_n = result[2]
        self.public_e = result[3]
        self.__private_key = result[4]
        
    def show(self):
        print(self.__key_pair, self.public_n, self.public_e, self.__private_key, sep = "\n")
        
    
    def RSA_decrypt(self, C):
        res = pow(C, self.__private_key, self.public_n)
        return res
    
    def RSA_sign(self, M):
        res = pow(M, self.__private_key, self.public_n)
        return res
    
    def Send_Key(self, e_1, n_1):
        while(self.public_n > n_1):
            print("Regenerating keys")
            print(self.public_n," && ", n_1)
            result = RSA_gen_constants()
            self.__key_pair = result[0], result[1]
            self.public_n = result[2]
            self.public_e = result[3]
            self.__private_key = result[4]
        
        k = random.randint(1, self.public_n-1)
        print("initial k, ", hex(k))
        S = pow(k, self.__private_key, self.public_n)
        S_1 = pow(S, e_1, n_1)
        k_1 = pow(k, e_1, n_1)
        
        return k_1, S_1
    
    def Receive_Key(self, k_1, S_1, e, n):
        k = pow(k_1, self.__private_key, self.public_n)
        S = pow(S_1, self.__private_key, self.public_n)
        
        if k == pow(S, e, n):
            __k = k
            print("k is authentic, ", hex(k))
        else:
            print("k is not authentic, try again")

## User initialization

In [237]:
A = User()
B = User()

def RSA_encrypt(m,e,n):
    res = pow(m,e,n)
    return res

def RSA_verify(S, M, e, n):
    res = pow(S,e,n)
    return res == M

In [272]:
print("Public n: ", A.public_n)
print("Public e: ", A.public_e)

Public n:  15115333322066772450441067451048002723717561506171604432016060143914893447974168884928991232487149882623226484531945047705152132156719173897180589227786310817
Public e:  65537


## Next corresponding functions are working with http://asymcryptwebservice.appspot.com/?section=rsa website and were created to check their corectness

## Server Key

In [259]:
server_key = int("8C9B7B4ADF8D45CF4CB63183681109D2D38B0941F6C04971FD3C0BC55CD99647", 16)
server_e = 2**16+1

## Encryption

In [240]:
M = int("12345", 16)
print("Public n: ",hex(A.public_n))
print("Public e: ",hex(A.public_e))
print("Decrypted M: ",hex(RSA_encrypt(M,server_e,server_key)))

0x14152bc92aaf35739d56c41679dabf19cfc123c8bce86e6a4b4b5e6330bd46fe4b49aa01a6efb6caf70d1d769460a1238ef382671da72b795414fa5927bc31ce363d
0x10001
0x40aa16f4b83ec117d98791751a04c63037e7eda0f4ca4629e700f3997e01118


## Decryption

In [241]:
C = int("028CA31431ED51576FFA245A3E3A044287233FDFD2C292D15FA7FDAEC4F001D0BDE8B684084CA914FE6B878A01C00DE0AC4AC1E227982C91D5B7E62036D49D5C9106", 16)
print("Encrypted M: ", hex(A.RSA_decrypt(C)))

0x11300296ca1f8491ce38eb8f8777f71f43ecf057f77333ec96cbbfacd11918201d346c96b2f24f23da60f22bdd0d85cf76edbae750dcda4709b6790a26fb6b9bff41


## Signature

In [264]:
M = int("12345", 16)
server_S = int("36880956AD2EDC33F2C8B731AA6823556FDE2F007A261824908F61144D10D1FC",16)
print("Verification (True/False): ",RSA_verify(server_S, M, server_e, server_key))

True


## Verification

In [243]:
M = int("1234", 16)
print(hex(A.RSA_sign(M)))
print(hex(A.public_n))
print(hex(A.public_e))
# result is presented on the website

0x2adb328def2857f8a4ef0d0237734f0674d191a21e1847c3188dad4b80e32f6afe65c7458d9c65ecdc4adfab8b77bd781eb55b46a0583ce93b42e822148243225b7
0x14152bc92aaf35739d56c41679dabf19cfc123c8bce86e6a4b4b5e6330bd46fe4b49aa01a6efb6caf70d1d769460a1238ef382671da72b795414fa5927bc31ce363d
0x10001


## Send key


In [260]:
print(hex(A.public_n))
print(hex(A.public_e))

0x4675a62425ad54847f38856b4cd386dd6dbe0eb36f48e0f52dbea598ec1c4d45374699b5db9f7240dd7cf53883d54c1c5ff59a66533244a3c7852990ef4aafb14a1
0x10001


In [267]:
server_encrypted_key = int("01D46C75E2CF703ACB160AE5615901A31409C708BB76E09FF4FF6E29EEBA497F6C2DE29E8D3F7D8193967DDDEACB0764F18D044E40C1E44C16DFEB5CE36C9CF2CF39", 16)
server_encrypted_signature = int("488AF41BAB8BD994803C9557808BD8723A9CB4E46109D15B88D8D05A83460EFABB5554453C0BD5C4E6E9AB440C00F2181ABF770664578965A99EEF5DF98C0D1F37",16)
A.Receive_Key(server_encrypted_key,server_encrypted_signature, A.public_e, server_key)

k is authentic,  0x5bad7deb572049f3


## Recieve Key

### для роботи даної функції потрібно збільшити модуль сервера з яким ми спілкуємось, а бо ж зменшити ключ абонента, що робиться трохи вище

In [268]:
server_key = int("87AE7AEB5B6DB02F738224DC2CB927132FC7E0D2ED094EFBCE540B4622053BE77E36B33C72904CE80E5493607681BF6EF7DBE5A3994B045059CD81DBF7EEA2D82AC260E6B1840CCFCD429850EA453BEC595A7B74DBFFD65F26EAF7EFCA4DA6238F80AFE8D2F2372523B44C1F1F97505FA3D0ACD1715DD5BCEB45C475B9FDF693", 16)
server_e = 2**16+1
k_1, s_1 = A.Send_Key(server_e, server_key)
print(hex(k_1))
print(hex(s_1))
print(hex(A.public_n))
print(hex(A.public_e))
# result is presented on the website

initial k,  0x1d329d00ec9a74f59e1cd2cbf5cb6246848709bf912c7ce8b37769d1e4aaec14e04e59e1d5cd6a0cfa64c4e4f515313b1932043a33128dc17bd1291ba5f6010b2ce
0x40c7ba3430d60eddb6b2384ca9ceb60fb214295a6d008d6bcbe72988d4d7859ae63c937c9759dcf3e4d3292249b9cafff485486b4fb21293a2cdd56a0eed398778fb4af036e24a69d5e9f8621d96cac986a2caace823426f35f82ebff245db1542e2a96d7680cce89ca826f8690cfeaf1bbee520ae807ab4e92bacd45ac7e8f7
0x68fba1793983c8adcca06d6ff3098addb61a12e115f4d9286f4cc32e268fdb7609fedb717526f57acfa35dfc53dd196b84858a6116843858364c4665a03035008ac64de399517011f40d7c73886f6fb1996885a0519725c51666198a0e0439d078defe62bb3b01df1d1680fd36ab22eac9f3ae4e3936c17422228c5be522645e
0x4675a62425ad54847f38856b4cd386dd6dbe0eb36f48e0f52dbea598ec1c4d45374699b5db9f7240dd7cf53883d54c1c5ff59a66533244a3c7852990ef4aafb14a1
0x10001


## Conclusion

По ходу виконання роботи не виникало значних труднощів.

Опрацювання алгоритмів і логіки працювання RSA були цікавими в реалізації й на невеликих числах, як виявилось достатньо швидким.

Алгоритм устворення *кращих* простих чисел виявився досить непердбачуваним так як $k$ у формулі $2kp+1$ могло досягати великих значень й збільшувати нове просте число майже вдвічі. (за кількістю бітів)

Лабораторна робота була не складною але цікавою, RSA ефективним і повідомлення захищеними :)